# Setup

In [1]:
from itertools import combinations, product
import time

# Requirements

# Pattern Gen

In [2]:
class PatternDatabase:
    def __init__(self):
        self.patterns = {}

    def add_pattern(self, name, implementation):
        if name not in self.patterns:
            self.patterns[name] = []
        self.patterns[name].append(implementation)

    def get_patterns(self, name):
        return self.patterns.get(name, [])

In [3]:
relationships = [
    {
        'output': 'cout',
        'operation': 'AND',
        'inputs': ['a', 'b']
    },
    {
        'output': 'sum',
        'operation': 'XOR',
        'inputs': ['a', 'b']
    }
]

In [4]:
def evaluate_expression(expr, input_map):
    """
    Evaluate the logic expression based on the input values.
    """
    # Replace inputs with their values
    eval_expr = expr
    for var, val in input_map.items():
        eval_expr = eval_expr.replace(var, str(val))
    # Replace logic operators with Python equivalents
    eval_expr = eval_expr.replace('~', ' not ')
    eval_expr = eval_expr.replace('nand', ' not (')
    eval_expr = eval_expr.replace('nor', ' not (')
    eval_expr = eval_expr.replace('&', ' and ')
    eval_expr = eval_expr.replace('|', ' or ')
    eval_expr = eval_expr.replace('^', ' != ')
    eval_expr = eval_expr.replace(')', ')')  # Fix for 'nand' and 'nor'
    # Evaluate the expression
    try:
        result = eval(eval_expr)
        print(eval_expr, result)
        return int(bool(result))
    except Exception as e:
        return None

In [5]:
def apply_gate(gate, left_expr, right_expr=None):
    """
    Create a new expression by applying a gate to one or two expressions.
    """
    if gate == '~':  # NOT gate
        return f"~({left_expr})"
    elif gate in ['nand', 'nor']:
        return f"{gate}({left_expr}, {right_expr})"
    else:
        return f"({left_expr} {gate} {right_expr})"

In [6]:
def is_valid_expression(expr, inputs):
    """
    Check if the expression correctly implements XOR.
    """

    for values in product([0, 1], repeat=len(inputs)):
        input_map = dict(zip(inputs, values))
        expected = input_map['a'] ^ input_map['b']
        result = evaluate_expression(expr, input_map)
        if result != expected:
            return False
    return True

In [7]:
def generate_expressions(inputs, constants, gate_set, max_gates):
    """
    Generate all logic expressions up to max_gates using the provided gate_set.
    """
    from itertools import combinations_with_replacement, product

    expressions_by_gate_count = {}
    # Gate count 0: the inputs and constants
    expressions_by_gate_count[0] = set(inputs + constants)
    valid_expressions = []

    for gate_count in range(1, max_gates + 1):
        expressions_by_gate_count[gate_count] = set()
        # Combine expressions from all lower gate counts
        for gc_left in range(gate_count):
            gc_right = gate_count - gc_left - 1
            for left_expr in expressions_by_gate_count[gc_left]:
                for right_expr in expressions_by_gate_count[gc_right]:
                    for gate in gate_set:
                        if gate == '~':  # Unary gate
                            new_expr = apply_gate(gate, left_expr)
                            expr_key = new_expr
                        else:
                            # Avoid duplicate expressions due to commutativity
                            if left_expr > right_expr and gate in ['&', '|', '^']:
                                continue
                            new_expr = apply_gate(gate, left_expr, right_expr)
                            expr_key = new_expr
                        if expr_key not in expressions_by_gate_count[gate_count]:
                            expressions_by_gate_count[gate_count].add(expr_key)
                            if is_valid_expression(new_expr, inputs):
                                valid_expressions.append((gate_count, new_expr))

    return valid_expressions

def count_gates(expr):
    """
    Count the number of gates in an expression.
    """
    # Simplistic gate count based on operator occurrences
    gate_symbols = ['&', '|', '^', '~', 'nand', 'nor']
    count = 0
    for gate in gate_symbols:
        count += expr.count(gate)
    return count

In [8]:
def generate_xor_implementations():
    inputs = ['a', 'b']
    constants = ['0', '1']
    max_gates = 2
    gate_set = ['&', '|', '^', '~', 'nand', 'nor']
    expressions = generate_expressions(inputs, constants, gate_set, max_gates)

    # Remove duplicates and sort by complexity
    unique_expressions = {}
    for gate_count, expr in expressions:
        if expr not in unique_expressions:
            unique_expressions[expr] = gate_count

    # Prepare the list and sort
    valid_expressions = [(gc, expr) for expr, gc in unique_expressions.items()]
    valid_expressions.sort(key=lambda x: (x[0], x[1]))  # Sort by gate count and expression

    return valid_expressions

# Generate implementations
xor_implementations = generate_xor_implementations()
for complexity, impl in xor_implementations:
    print(f"Complexity: {complexity}, Implementation: {impl}")

(0  and  0) 0
(0  and  0) 0
(0  or  0) 0
(0  or  0) 0
(0  !=  0) False
(0  !=  0) False
 not (0) True
(0  and  0) 0
(0  and  1) 0
(0  or  0) 0
(0  or  1) 1
(1  or  0) 1
(1  or  1) 1
(0  !=  0) False
(0  !=  1) True
(1  !=  0) True
(1  !=  1) False
(1  and  0) 0
(1  and  0) 0
(1  or  0) 1
(1  !=  0) True
 not (1) False
 not (1) False
(1  and  1) 1
(1  or  1) 1
(1  !=  1) False
(1  !=  1) False
(1  and  0) 0
(1  and  1) 1
(1  and  0) 0
(1  or  0) 1
(1  !=  0) True
 not (0) True
(0  and  0) 0
(1  and  1) 1
(0  and  0) 0
(0  or  0) 0
(1  or  1) 1
(0  or  0) 0
(0  !=  0) False
(1  !=  1) False
(0  and  0) 0
(0  and  0) 0
(0  or  0) 0
(0  or  0) 0
(0  !=  0) False
(0  !=  0) False
 not (0) True
(0  and  1) 0
(0  and  1) 0
(0  or  1) 1
(0  !=  1) True
(0  and  0) 0
(0  and  1) 0
(0  or  0) 0
(0  or  1) 1
(0  or  0) 0
(0  !=  0) False
(0  !=  1) True
(0  !=  0) False
(0  and  0) 0
(0  and  0) 0
(0  or  0) 0
(0  or  0) 0
(0  !=  0) False
(0  !=  0) False
 not (0) True
(0  and  n0nd(0, 0)) 0
(0 

In [ ]:
# Initialize pattern database
pattern_db = PatternDatabase()

# Add multiple patterns for XOR gate
pattern_db.add_pattern('XOR', '({a} & ~{b}) | (~{a} & {b})')
pattern_db.add_pattern('XOR', '~{a} & {b} | {a} & ~{b}')
pattern_db.add_pattern('XOR', '{a} ^ {b}')
pattern_db.add_pattern('XOR', '~({a} & {b}) & ({a} | {b})')

# Add multiple patterns for AND gate
pattern_db.add_pattern('AND', '{a} & {b}')
pattern_db.add_pattern('AND', '~(~{a} | ~{b})')
pattern_db.add_pattern('AND', 'nand(~{a}, ~{b})')  # Assuming nand is defined


# Module Gen

In [ ]:
from itertools import product

class ModuleGenerator:
    def __init__(self, pattern_db):
        self.pattern_db = pattern_db
        self.generated_modules = {}

    def generate_module_variations(self, module_name, relationships, submodules=None):
        # For each relationship, get all patterns
        patterns_list = []
        for rel in relationships:
            patterns = self.pattern_db.get_patterns(rel['operation'])
            if not patterns:
                patterns = [self.default_pattern(rel['operation'])]
            patterns_list.append(patterns)
        # Generate all combinations
        variations = []
        for pattern_combo in product(*patterns_list):
            verilog_code = self.generate_module_with_patterns(module_name, relationships, pattern_combo, submodules)
            variations.append(verilog_code)
        return variations

    def generate_module_with_patterns(self, module_name, relationships, patterns, submodules=None):
        # Similar to generate_module but uses provided patterns
        verilog_code = f"module {module_name} (\n"
        # Collect inputs and outputs
        inputs = set()
        outputs = set()
        wires = set()
        for rel in relationships:
            outputs.add(rel['output'])
            inputs.update(rel['inputs'])
        if submodules:
            for sub in submodules:
                inputs.update(sub.get('inputs', []))
                outputs.update(sub.get('outputs', []))
                wires.update(sub.get('wires', []))
        # Define inputs and outputs
        verilog_code += ",\n".join([f"    input wire {i}" for i in inputs]) + ",\n"
        verilog_code += ",\n".join([f"    output wire {o}" for o in outputs]) + "\n);\n"
        # Define wires
        for w in wires:
            verilog_code += f"    wire {w};\n"
        # Add assignments
        for idx, rel in enumerate(relationships):
            expr = patterns[idx].format(**{k: k for k in rel['inputs']})
            verilog_code += f"    assign {rel['output']} = {expr};\n"
        verilog_code += "endmodule\n"
        return verilog_code

    def default_pattern(self, operation):
        # Default patterns if none are provided
        op_map = {'AND': '&', 'OR': '|', 'XOR': '^'}
        return f" {{ {f' {op_map.get(operation, operation)} '.join(['{{{}}}'.format(i) for i in range(2)])} }} "


In [ ]:
module_gen = ModuleGenerator(pattern_db)
variations = module_gen.generate_module_variations('half_adder', relationships)

for idx, variant in enumerate(variations):
    print(f"// Variation {idx+1}\n{variant}\n")

# Formal Verification

In [ ]:
class VerificationGenerator:
    def generate_sby_file(self, module_name, properties):
        sby_content = f"""
[options]
mode prove

[engines]
smtbmc

[script]
read -formal {module_name}.v
prep -top {module_name}

[files]
{module_name}.v

[prove]
"""
        # Add properties
        for prop in properties:
            sby_content += prop + "\n"
        # Write to file
        with open(f"{module_name}.sby", "w") as f:
            f.write(sby_content)

    def generate_properties(self, module_name, relationships):
        properties = []
        for rel in relationships:
            input_expr = ' ^ '.join([f"{i}" for i in rel['inputs']]) if rel['operation'] == 'XOR' else ' & '.join([f"{i}" for i in rel['inputs']])
            properties.append(f"assert property ({rel['output']} == ({input_expr}));")
        return properties

In [ ]:
verifier = VerificationGenerator()
properties = verifier.generate_properties('half_adder', relationships)
verifier.generate_sby_file('half_adder', properties)